In [1]:
import numpy as np
import ufl

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx import mesh, fem, plot, io
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import DiagonalType
# import gmsh

import pyvista

In [95]:
L = 2.5
H = 0.2
Nx = 20
Ny = 5
# mesh = RectangleMesh(Point(0., 0.), Point(L, H), Nx, Ny, "crossed")

In [96]:
fe_mesh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                             points=((0.0, 0.0), (L, H)), n=(Nx, Ny),
                             cell_type = mesh.CellType.quadrilateral, 
                             diagonal = DiagonalType.crossed)#

In [97]:
def eps(u):
    return ufl.sym(ufl.grad(u))

In [98]:
E = 2.1e11;
ν = 0.3;
model = "plane_stress";

μ = E / ( 2 * (1 + ν) );
λ = E*ν / ( (1 + ν) * (1 - 2*ν) );
if model == "plane_stress":
    λ = 2*μ*λ/(λ+2*ν);
def sigma(u):
    ε = eps(u);
    return λ * ufl.tr(ε) * ufl.Identity(2) + \
            2.0 * μ * ε

# Weak formulation
In the Dolphinx the second parameter of "VectorFunctionSpace" is a tuple, which define all properties of elements, such basicfunction family and degree of polynomials. Or it Could be an object of type ElementMetaData

In [99]:
ρ = 7800;g =9.8; 
f = fem.Constant(fe_mesh, (0, -ρ*g));

V = fem.VectorFunctionSpace(fe_mesh, ("CG", 2))
du = ufl.TrialFunction(V);
u_ = ufl.TestFunction(V);
a = ufl.inner(sigma(du), eps(u_))*ufl.dx
l = ufl.inner(f, u_)*ufl.dx

# Problem solving

In [101]:
def left(x):
    return np.isclose(x[0], 0.)

fdim = fe_mesh.topology.dim - 1
left_facets = mesh.locate_entities_boundary(fe_mesh, fdim, left)

u_D = np.array([0.,0.], dtype=ScalarType)
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, left_facets), V)

In [102]:
problem = fem.petsc.LinearProblem(a, l, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

# Visualization

In [104]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()

topology, cell_types, geometry = plot.create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
grid.point_data["u"] = np.c_[uh.x.array.reshape((geometry.shape[0], 2)), np.zeros(geometry.shape[0]).T]
actor_0 = plotter.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=500)
actor_1 = plotter.add_mesh(warped, show_edges=True)

plotter.show_axes()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [105]:
with io.XDMFFile(fe_mesh.comm, "deformation.xdmf", "w") as xdmf:
    xdmf.write_mesh(fe_mesh)
    uh.name = "Deformation"
    xdmf.write_function(uh)